In [1]:
import os
import pandas as pd
from QUANTAXIS.QAUtil.QADate import QA_util_datetime_to_strdate as date_to_str
from QUANTAXIS import QA_fetch_index_day_adv
from QUANTAXIS import QA_fetch_stock_day_adv
from QUANTAXIS import QA_fetch_stock_list
import pyfolio as pf
import numpy as np
import jupyter_helper

C:\Users\GuQiang\AppData\Local\conda\conda\envs\finance35\lib\site-packages\empyrical\utils.py:32: UserWarning: Unable to import pandas_datareader. Suppressing import error and continuing. All data reading functionality will raise errors; but has been deprecated and will be removed in a later version.
  warnings.warn(msg)


## 读取所有股票的年度分析数据

In [2]:
lst=QA_fetch_stock_list()

s,e=jupyter_helper.get_start_end_date()

jupyter_helper.get_year_stats(date_to_str(s),date_to_str(e),lst=lst.code.values)

QA Error QA_fetch_stock_day_adv parameter code=002946 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=002947 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300755 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300758 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=300759 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=600928 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=601298 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None
QA Error QA_fetch_stock_day_adv parameter code=601615 , start=2005-04-08, end=2019-01-18 call QA_fetch_stock_day return None


Annual return  Cumulative returns  Annual volatility  \
code   year                                                         
000001 2005  -1.416625e-01           -0.095729           0.379542   
       2006   1.624502e+00            1.304140           0.428276   
       2007   2.659417e+00            2.009213           0.606860   
       2008  -6.884859e-01           -0.675236           0.662368   
       2009   1.728770e+00            1.509784           0.483964   
       2010  -4.151215e-01           -0.334036           0.369005   
       2011  -2.875499e-02           -0.026841           0.267774   
       2012   7.036977e-02            0.064608           0.234383   
       2013   2.546339e-01            0.236693           0.495160   
       2014   6.023672e-01            0.575611           0.330486   
       2015  -9.721348e-02           -0.093910           0.451552   
       2016  -2.265229e-02           -0.021852           0.184491   
       2017   4.944744e-01            0.473183           0.259950   
       2018  -3.149158e-01           -0.304556           0.337297   
000002 2005   2.475648e-01            0.142728           0.360326   
       2006   2.711342e+00            2.397115           0.458532   
       2007   2.031471e+00            1.812943           0.633833   
       2008  -6.576587e-01           -0.644298           0.663328   
       2009   6.571519e-01            0.621015           0.448559   
       2010  -2.269372e-01           -0.216604           0.361232   
       2011  -1.461632e-01           -0.140253           0.267230   
       2012   4.308010e-01            0.400614           0.288226   
       2013  -2.925782e-01           -0.266858           0.378326   
       2014   8.721901e-01            0.835287           0.358084   
       2015   7.637210e-01            0.693666           0.494522   
       2016  -5.369442e-02           -0.026365           0.536865   
       2017   5.897042e-01            0.549288           0.383345   
       2018  -2.483905e-01           -0.239826           0.423353   
000004 2005  -3.748734e-01           -0.283737           0.503629   
       2006   2.749058e-01            0.223150           0.425426   
...                    ...                 ...                ...   
603997 2017  -1.411361e-01           -0.133846           0.353427   
       2018  -4.573666e-01           -0.308390           0.387564   
603998 2005            NaN                 NaN                NaN   
       2006            NaN                 NaN                NaN   
       2007            NaN                 NaN                NaN   
       2008            NaN                 NaN                NaN   
       2009            NaN                 NaN                NaN   
       2010            NaN                 NaN                NaN   
       2011            NaN                 NaN                NaN   
       2012            NaN                 NaN                NaN   
       2013            NaN                 NaN                NaN   
       2014   1.100145e+03            0.649205           1.137010   
       2015   1.147799e+00            0.949107           0.721016   
       2016   1.618423e-01            0.155635           0.520462   
       2017  -3.862024e-01           -0.375409           0.293731   
       2018  -6.853609e-01           -0.579893           0.470606   
603999 2005            NaN                 NaN                NaN   
       2006            NaN                 NaN                NaN   
       2007            NaN                 NaN                NaN   
       2008            NaN                 NaN                NaN   
       2009            NaN                 NaN                NaN   
       2010            NaN                 NaN                NaN   
       2011            NaN                 NaN                NaN   
       2012            NaN                 NaN                NaN   
       2013            NaN                 NaN                NaN   
       2014

limit_output extension: Maximum message size of 10000 exceeded with 25912 characters

## 读取指定股票的年度分析数据

In [4]:
jupyter_helper.calc_perf_stats_by_year('000002',jupyter_helper.get_benchmark_code(),s,e)

Annual return  Cumulative returns  Annual volatility  \
code   year                                                         
000002 2005       0.247565            0.142728           0.360326   
       2006       2.711342            2.397115           0.458532   
       2007       2.031471            1.812943           0.633833   
       2008      -0.657659           -0.644298           0.663328   
       2009       0.657152            0.621015           0.448559   
       2010      -0.226937           -0.216604           0.361232   
       2011      -0.146163           -0.140253           0.267230   
       2012       0.430801            0.400614           0.288226   
       2013      -0.292578           -0.266858           0.378326   
       2014       0.872190            0.835287           0.358084   
       2015       0.763721            0.693666           0.494522   
       2016      -0.053694           -0.026365           0.536865   
       2017       0.589704            0.549288           0.383345   
       2018      -0.248390           -0.239826           0.423353   

             Sharpe ratio  Calmar ratio  Stability  Max drawdown  Omega ratio  \
code   year                                                                     
000002 2005      0.792542      0.912895   0.432380     -0.271186     1.145253   
       2006      3.092249     10.572720   0.705576     -0.256447     1.679638   
       2007      2.067398      6.471218   0.860785     -0.313924     1.403675   
       2008     -1.281374     -0.895960   0.860150     -0.734027     0.819245   
       2009      1.349868      1.880365   0.569038     -0.349481     1.263235   
       2010     -0.533083     -0.624532   0.039828     -0.363371     0.910384   
       2011     -0.457579     -0.642819   0.524660     -0.227378     0.925526   
       2012      1.386984      2.052028   0.217157     -0.209939     1.264063   
       2013     -0.725781     -0.779754   0.793176     -0.375218     0.883479   
       2014      1.929429      4.907605   0.870690     -0.177722     1.407044   
       2015      1.392347      3.783083   0.365053     -0.201878     1.273973   
       2016      0.158409     -0.200594   0.313336     -0.267677     1.030654   
       2017      1.396858      3.379950   0.789908     -0.174471     1.320980   
       2018     -0.463375     -0.504011   0.594166     -0.492828     0.926086   

             Sortino ratio      Skew  Kurtosis  Tail ratio  \
code   year                                                  
000002 2005       1.189484  0.203426  1.371047    1.157820   
       2006       5.713021  0.574761  0.760630    1.477692   
       2007       3.307137  0.102415  0.369599    1.528614   
       2008      -1.720856  0.092179 -0.278274    0.943801   
       2009       2.088511  0.113761  1.460199    1.179810   
       2010      -0.769730  0.417072  2.770292    1.016349   
       2011      -0.623653 -0.233069  1.223881    1.024921   
       2012       2.164537  0.088678  0.914225    1.257075   
       2013      -1.028689  0.098257  2.061875    1.392776   
       2014       3.398934  0.787619  2.879390    1.815147   
       2015       2.267164  0.472486  1.799093    1.311494   
       2016       0.261324  0.949148  2.510994    1.732992   
       2017       2.605061  1.414937  4.838696    1.401044   
       2018      -0.658318  0.138263  1.016388    1.112417   

             Daily value at risk     Alpha      Beta  
code   year                                           
000002 2005            -0.044264  0.341054  1.037877  
       2006            -0.052143  0.319475  1.244536  
       2007            -0.074656  0.102848  1.141843  
       2008            -0.086944  0.292898  1.093733  
       2009            -0.054110 -0.131836  1.069101  
       2010            -0.046275 -0.136774  0.972766  
       2011            -0.034153  0.156493  0.934324  
       2012            -0.034727  0.308393  0.953089  
       2013            -0.048754 -0.151970  1.199315  
       2014  